In [1]:
## NowPlaying Cultural Sentiment Data
"""
cs_header = ["hashtag", "AFINN_min", "AFINN_max", "AFINN_sum", "AFINN_avg", "OpinionLex_min", "OpinionLex_max", "OpinionLex_sum", "OpinionLex_avg", "Sentistrength_min", "Sentistrength_max", "Sentistrength_sum", "Sentistrength_avg", "Vader_min", "Vader_max", "Vader_sum", "Vader_avg", "SentimentHashtag_min", "SentimentHashtag_max", "SentimentHashtag_sum", "SentimentHashtag_avg"]
np_cs = pd.read_csv('np_cultural_sentiment.csv',names = cs_header)
"""
#IMPORTING Cultural Sentiment Data to MongoDB
"""
np_cs_json = json.loads(np_cs.to_json(orient="records"))
db.cultural_sentiment.drop()
db.cultural_sentiment.insert_many(np_cs_json)
"""
## Transformation of NowPlaying Cultural Tweets to DataframeStructure
"""
output = []
with open("np_cultural.json") as f:
    for line in f:     
        output.append(json.load|s(line))
np_c = pd.DataFrame(output)
"""
#IMPORTING Cultural Tweets to MongoDB
"""
db.cultural_tweet.drop()
records = json.loads(np_c.T.to_json()).values()
db.cultural_tweet.insert_many(records)
"""

## Transformation and Importing of Playlist Data from NowPlaying to MongoDB
"""
np_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])
np_pl_json = json.loads(np_pl.to_json(orient="records"))
db.playlist.drop()
db.playlist.insert_many(np_pl_json)
"""

'\nnp_pl = pd.read_csv("playlist.csv", names=["playlist_id","track_id","playlist_name"])\nnp_pl_json = json.loads(np_pl.to_json(orient="records"))\ndb.playlist.drop()\ndb.playlist.insert_many(np_pl_json)\n'

# CULTURAL #NOWPLAYING DATA
*********************************

This dataset contains data describing the listening events of users (extracted from the #nowplaying dataset), the emotion extracted from the hashtags used in the according tweets and information about the location of the user.


TWITTER AND TRACK DATA
-------------------------------
The data regarding the listening events is contained in np_cultural.zip and is encoded as json. It holds the following information:

- id: the id of the underlying tweet [*]
- user_id: the id of the user who sent the tweet (MD5 of it)
- user_lang: The BCP 47 code for the user’s self-declared user interface language. [*]
- user_time_zone: [*]
- text: actual content of the tweet [*]
- tweet_lang: language of the tweet (as detected by Twitter; BCP 47 language identifier corresponding to the machine-detected language of the Tweet text, or und if no language could be detected.) [*]
- geo: Deprecated version of coordinates (however, we deal with data stemming from before this API change, therefore we still include it; cf. coordinates for a description)
- coordinates: Represents the geographic location of this Tweet as reported by the user or client application. The inner coordinates array is formatted as geoJSON (longitude first, then latitude). [*]
- place: When present, indicates that the tweet is associated (but not necessarily originating from) a Place. [*]
- created_at: time the tweet was sent. [*]
- source: Utility used to post the Tweet, as an HTML-formatted string. [*]
- track_title: title of the track the user tweeted about
- track_id: the unique id of the track (from #nowplaying dataset) 
- artist_name: name of artist performing the track
- artist_id: the unique id of the artist (from #nowplaying dataset) 
- hashtags: list of hashtags used in the tweet.

[*] for further information about the information gathered from Twitter, please consult https://dev.twitter.com/overview/api/tweets

Please note that we do only add key-value pairs for geo/coordinates/place information if this information was provided by the Twitter API (i.e., missing keys signal that this information is not available for the given tweet).


SENTIMENT DATA
-------------------------------
The data regarding the hashtag's sentiment (if any could be obtained) is contained in np_cultural_sentiment.csv and is formatted as csv. The sentiment score was obtained by applying a set of well-known sentiment dictionaries. The sentiment scores are scaled between 0 and 1 (very negative to very positive). For each dictionary, we list the minimum, maximum, sum and average sentiment score across all hashtags used within every tweet (most tweets only feature a single hashtag we can assign a sentiment value to)
It contains the following information (in this particular order):
- name of the hashtag
- AFINN dictionary (min, max, sum, avg)
- Opinion Lexicon (min, max, sum, avg)
- Sentistrength Lexicon (min, max, sum, avg)
- Vader (min, max, sum, avg)
- Sentiment Hashtag Lexicon (min, max, sum, avg)


Please note that we only added hashtags for which we could obtain a sentiment value from at least one sentiment dictionary.

In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np 
import json
import csv
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from difflib import SequenceMatcher


In [3]:
#Call Mongo Instance
client = MongoClient()
db = client.now_playing

#Spotify Instance
client_credentials_manager = SpotifyClientCredentials(client_id="7736d10450e04c5f9e302bb07a4f6cf7", client_secret="a11e29bc5c324ddeb19fc6249d303814",)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Importing the MusicBrainz Mapping of artists and tracks and tracks from cultural dataset

`mongoimport -d now_playing -c mb_tracks --type csv --file mb_tracks.csv --fields "np_id,mb_track_id"`

`mongoimport -d now_playing -c mb_artists --type csv --file mb_artists.csv --fields "np_id,mb_artist_id"`

`mongoimport -d now_playing -c np_cultural_mb_tracks --type csv --file np_cultural_mb_tracks.csv --headerline
`

In [4]:
user_ids = db.cultural_tweet.distinct("user_id")
type(user_ids)


list

In [6]:
#db.cultural_tweet.find({"hashtag": "nobeats"})
user_tweets = db.cultural_tweet.find({"user_id": '1c10f9788fdcc4baf6cf6a2631fe78bc12102418'})
#TOTAL Number of Tweets
print("Total number of Tweets: "+ str(db.cultural_tweet.find({}).count()))
print("Total number of Songs: " + str(len(db.cultural_tweet.distinct("track_title"))))
#Get Distinct User Ids
print("Total number of Users: " + str(len(user_ids)))
print("Total number of Tweets with duration: "+ str(db.cultural_tweet.find({"duration_ms" : {"$exists": True}}).count()))

Total number of Tweets: 564301
Total number of Songs: 39776
Total number of Users: 9431
Total number of Tweets with duration: 61013


In [7]:
first = user_ids[0]
user_songs = db.cultural_tweet.find({"user_id": first})
initial_song = user_songs[0]
initial_date = initial_song["created_at"]

In [8]:
#db.mb_tracks.find({"np_id": initial_song["track_id"]}).count()
#for song in user_songs:
#    print(str(song["track_title"]))
#    print(str(song["track_id"]))
    
print(len(db.cultural_tweet.distinct("track_id")))
print(len(db.cultural_tweet.distinct("track_title")))

51045
39776


In [9]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def convertMillis(millis):
     seconds = (millis/1000)%60
     minutes = (millis/(1000*60))%60
     return str(int(minutes)) + ":" +str(int(seconds))


In [78]:
"""
search_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()
results = sp.search(q=search_query,limit=5)
print("Looking for: "+search_query)
print("Querying...")
for track in results["tracks"]["items"]:
    artist_names = list(map(lambda x: x["name"],track["artists"]))
    print(" ".join(artist_names))
    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))
    print(convertMillis(track["duration_ms"]))
    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()
    print("Similarity Score: " + str((similar(search_query, result_matcher))))
"""

'\nsearch_query = initial_song["track_title"].lower() +" "+ initial_song["artist_name"].lower()\nresults = sp.search(q=search_query,limit=5)\nprint("Looking for: "+search_query)\nprint("Querying...")\nfor track in results["tracks"]["items"]:\n    artist_names = list(map(lambda x: x["name"],track["artists"]))\n    print(" ".join(artist_names))\n    print("Found: " + str(track["name"]) + " "+str(track["artists"][0]["name"]))\n    print(convertMillis(track["duration_ms"]))\n    result_matcher = track["name"].lower() + " " + track["artists"][0]["name"].lower()\n    print("Similarity Score: " + str((similar(search_query, result_matcher))))\n'

In [ ]:
final_result = dict()
#Get Spotify's song duration
track_list = db.cultural_tweet.distinct("track_id")
for track in track_list:
    song_name = db.cultural_tweet.find_one({"track_id": track }, {"track_title" : 1 , "artist_name": 1})
    search_query = song_name["track_title"].lower() + " " + song_name["artist_name"].lower()
    search_result = sp.search(q = search_query, limit= 3, type="track")
    for t in search_result["tracks"]["items"]:
        artists = " ".join(list(map(lambda x: x["name"],t["artists"])))
        result_matcher = t["name"].lower() + " " + artists.lower()
        similarity_score = similar(search_query,result_matcher)
        if(similarity_score > 0.95):
            print("Found " + result_matcher)
            print("Similarity Score: " + str(similarity_score))
            final_result[track] = t["duration_ms"]
            break

Found intuition jewel
Similarity Score: 1.0
Found jesus queen
Similarity Score: 1.0
Found deus the sugarcubes
Similarity Score: 1.0
Found gangsta kat dahlia
Similarity Score: 1.0
Found symbolic death
Similarity Score: 1.0
Found as stevie wonder
Similarity Score: 1.0
Found scala agoria
Similarity Score: 1.0
Found you are my sunshine james last
Similarity Score: 1.0
Found the way you love me faith hill
Similarity Score: 1.0
Found never be the same ulrich schnauss
Similarity Score: 1.0
Found 3 birds the dead weather
Similarity Score: 1.0
Found amapola james last
Similarity Score: 1.0
Found towers little mix
Similarity Score: 1.0
Found isaiah boar
Similarity Score: 1.0
Found kosmos yob
Similarity Score: 1.0
Found pretty noose soundgarden
Similarity Score: 1.0
Found volare gipsy kings
Similarity Score: 1.0
Found where do you sleep at night lynch mob
Similarity Score: 1.0
Found the state of madness madness of the night
Similarity Score: 1.0
Found tower zola jesus
Similarity Score: 1.0
Found 

In [82]:
for track in final_result:
    db.cultural_tweet.update_many({"track_id": track},
                            {"$set" : {
                                "duration_ms" : final_result[track]
                            }})
    


KeyboardInterrupt: 

{'_id': ObjectId('59bfff98ed415cd14d0d27ea'),
 'artist_id': '029796844c2ca8bc9380dc0a046a6223',
 'artist_name': 'James Last',
 'coordinates': None,
 'created_at': '2014-03-04 11:22:52',
 'duration_ms': 182693,
 'geo': None,
 'hashtags': ['radionomy'],
 'id': 440794499095478300,
 'place': None,
 'source': 'Share.Radionomy.com',
 'text': '#NowPLaying James Last - Amapola http://t.co/ucxjELVE5a #ListenLive #radionomy #Instrumental',
 'track_id': '0669adeb25a376ad2219737816de38cf',
 'track_title': 'Amapola',
 'tweet_lang': 'sk',
 'user_id': 'a4efac8193cba040f6019100343944a59d837f1',
 'user_lang': 'nl',
 'user_location': '',
 'user_time_zone': None}

In [11]:
Analyze the music listening behavior of people on Twitter and analyze the emotion in tweets 
they posted while listening to this music

SyntaxError: invalid syntax (<ipython-input-11-8ae5e2d82f58>, line 1)